In [10]:
"""in the no-robot condition don't run this script"""
import threading
import time
import datetime
import qi

import sys
import os.path
sys.path.insert(0, '../../db')

import database as db

import bisect

class MemoryRobot:
    
    def __init__(self, p_name):
        
        self.setPerson(p_name)
        
#         # Connect to robot, and set configurations
#         self.robot_ip = "192.168.1.11"
#         self.robot_port = 9559
#         connect = self.connectToRobot(self.robot_ip)
#         if connect == -1:
#             # TODO: TRY THIS AND ADD STH TO NOTIFY THE TABLET!!!
#             sys.exit(1)
        self.past_holiday = ""
        self.came_early = False
        self.missed_sessions = []
        self.on_schedule = self.isPatientOnSchedule()
        
    def connectToRobot(self, ip, port=9559):
        self.robot_ip = ip
        self.robot_port = port
        self.session = qi.Session()
        try:
            self.session.connect("tcp://" + ip + ":" + str(port))
        except RuntimeError:
            print ("Can't connect to Naoqi at ip \"" + ip + "\" on port " + str(port) +".\n"
               "Please check your script arguments. Run with -h option for help.")
            return -1
        self.animatedSpeechProxy = self.session.service("ALAnimatedSpeech")
        self.tts = self.session.service("ALTextToSpeech")
        self.configuration = {"bodyLanguageMode":"contextual"}
        self.tts.setVolume(0.4)
    
    def say(self, sentence):
        threading.Thread(target = self.animatedSpeechProxy.say, args=(sentence,self.configuration)).start()
        
    def loadInfo(self):
        self.db_handler = db.DbHandler()
        p = self.db_handler.get_person_data({"name":self.p_name})
        self.p_gender = str(p["gender"])
        self.p_age = int(p["age"])
        self.p_height = float(p["height"])
        self.p_times = []
        for tt in p["times"]:
            self.p_times.append([tt.time().strftime('%H:%M:%S'), tt.isoweekday()])        
        
    def loadSessionData(self):
        s = self.db_handler.get_all_sessions({"patient":self.p_name})
        counter = 0
        self.p_dates = []
        self.p_events = []
        self.p_averages = []
        for ss in s:
            self.p_dates.append(datetime.datetime.strptime(ss["date"], '%Y-%m-%d %H:%M:%S.%f'))
#             print datetime.datetime.strptime(ss["date"], '%Y-%m-%d %H:%M:%S.%f').isoweekday()
#             print datetime.datetime.strptime(ss["date"], '%Y-%m-%d %H:%M:%S.%f').time().strftime('%H:%M:%S')
            s_events = []
            e = ss["events"]
            for e_s in e:
                s_events.append(e_s)
            self.p_events.append(s_events)
            s_avg = []
            a = ss["average"]
            for a_s in a:
                s_avg.append(a_s)
            self.p_averages.append(s_avg)    
            counter = counter + 1
            
        self.p_num_sessions = counter
        
    def setPerson(self, p_name):
        self.p_name = p_name
        self.loadInfo()
        self.loadSessionData()
    
    def getExpectedSession(self, sorted_times, session_date, session_day = None, session_index = None):
        if session_day is None:
            session_day = session_date.isoweekday()
        if session_index is None:
            session_index = sorted_times.index(session_day)
        if session_index < len(sorted_times) - 1:
            day_dif = sorted_times[session_index + 1] - session_day 
        else:
            day_dif = 7 - session_day + sorted_times[0]
            self.past_holiday = "weekend"
        expected_session = session_date + datetime.timedelta(days=day_dif)
        return expected_session
        
    def isPatientOnSchedule(self):
        """check if the missing session is on bank holiday"""
        
        on_schedule = True
        
        colombia_holidays_2017 = [datetime.date(2017, 1, 1), datetime.date(2017, 1, 9), 
                                  datetime.date(2017, 3, 20), datetime.date(2017, 4, 13),
                                  datetime.date(2017, 4, 14), datetime.date(2017, 5, 1),
                                  datetime.date(2017, 5, 29), datetime.date(2017, 6, 19),
                                  datetime.date(2017, 6, 26), datetime.date(2017, 7, 3),
                                  datetime.date(2017, 7, 20), datetime.date(2017, 8, 7),
                                  datetime.date(2017, 8, 15), datetime.date(2017, 10, 16),
                                  datetime.date(2017, 11, 6), datetime.date(2017, 11, 13),
                                  datetime.date(2017, 11, 13), datetime.date(2017, 12, 8),
                                  datetime.date(2017, 12, 25)]
        
        # TODO: look at the last session date, look at the current date, compare the current date to the 
        # date that the session should have happened, compare the date that should have happened to holiday 
        # and find if the patient skipped a day that wasnt a holiday
        if self.p_times >= 2 and self.p_num_sessions >= 2:
            #TODO: correct these
            last_session = self.p_dates[-1].date() + datetime.timedelta(days=1)
            print last_session
            
            cur_session = datetime.datetime.now().date() + datetime.timedelta(days=11)
            print cur_session
            
            sorted_times = sorted(self.p_times,key=lambda l:l[1])
            sorted_times = [row[1] for row in sorted_times]

            last_session_day = last_session.isoweekday()
            if last_session_day in sorted_times:
                last_session_index = sorted_times.index(last_session_day)
                expected_session = self.getExpectedSession(sorted_times, last_session, last_session_day, last_session_index)
            else:
                last_session_index = bisect.bisect(sorted_times, last_session_day)
                expected_session = self.getExpectedSession(sorted_times, last_session, last_session_day, last_session_index-1)     
            
            if expected_session in colombia_holidays_2017:
                self.past_holiday = "bank holiday"
                expected_session = self.getExpectedSession(expected_session)
                print expected_session            
                
            if cur_session > expected_session:
                # TODO: missed a few sessions
                self.missed_sessions.append(expected_session)
                missed = cur_session - expected_session
                while missed.days > 0:                  
                    expected_session = self.getExpectedSession(sorted_times, expected_session)
                    if cur_session > expected_session:
                        self.missed_sessions.append(expected_session)
                        missed = cur_session - expected_session
                    else:
                        break
                print self.missed_sessions
                on_schedule = False
            elif cur_session < expected_session:
                self.came_early = True
            print on_schedule
        return on_schedule

        
if __name__ == "__main__":
    MR = MemoryRobot("sergio sierra")
    

2017-04-25
2017-05-05
[datetime.date(2017, 4, 26), datetime.date(2017, 4, 29), datetime.date(2017, 5, 1), datetime.date(2017, 5, 3)]
False
